In [1]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
from random import randint

In [1]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('csv', data_files={
    'train' : 'pelota_plata_train.csv',
    'test' : 'pelota_plata_test.csv'
})

Using custom data configuration default-964874b90c883f0d
0 tables [00:00, ? tables/s]Downloading and preparing dataset csv/default (download: Unknown size, generated: Unknown size, post-processed: Unknown size, total: Unknown size) to C:\Users\mv502\.cache\huggingface\datasets\csv\default-964874b90c883f0d\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0...
Dataset csv downloaded and prepared to C:\Users\mv502\.cache\huggingface\datasets\csv\default-964874b90c883f0d\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [5]:
type(dataset)

datasets.dataset_dict.DatasetDict

## Leemos los datos

In [10]:
csv = pd.read_csv('palabras_corregidas_secundaria.csv', encoding='latin1')

## Ecojemos la pregunta *Pelota de Pelos*

In [11]:
datos = pd.DataFrame({
    'sentence' : csv.iloc[:,4],
    'label' : csv.iloc[:,5]
})
datos

,sentence,label
0,que es dura o aguanta,1
1,puede ser una medalla o una esfera color metál...,1
2,una joya,1
3,seria muy brillosa la pelota,2
4,da a conocer que que es uno muy fuerte,3
...,...,...
465,pues tal vez la medalla esta llena de plata,1
466,yo puse que la frase se podría referir a la lu...,3
467,que es brillante,3
468,porque ya establecimos que a todo le esta dici...,1


## Convertimos los 3 en 1 y todo lo demás en 0, para hacer una predicción binaria

In [15]:
datos['label'] = [1 if i == 3 else 0 for i in datos['label'].tolist()]
datos

,sentence,label
0,que es dura o aguanta,0
1,puede ser una medalla o una esfera color metál...,0
2,una joya,0
3,seria muy brillosa la pelota,0
4,da a conocer que que es uno muy fuerte,1
...,...,...
465,pues tal vez la medalla esta llena de plata,0
466,yo puse que la frase se podría referir a la lu...,1
467,que es brillante,1
468,porque ya establecimos que a todo le esta dici...,0


## Importamos el tokenizer y el modelo

In [42]:
num_labels = 2
model_checkpoint = 'datificate/gpt2-small-spanish'
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)
model = GPT2ForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Some weights of the model checkpoint at datificate/gpt2-small-spanish were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at datificate/gpt2-small-spanish and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### TrainingArguments es una clase para los argumentos del entrenamiento

In [101]:
args = TrainingArguments(
    'prueba-gpt2',
    evaluation_strategy='epoch',
    learning_rate = 2e-5,
    # per_device_train_batch_size= ------------- opcional
    # per_device_eval_batch_size = ------------ opcional
    num_train_epochs = 5,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    # metric_for_best_model = 
)

C:\Users\mv502\anaconda3\envs\tf\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Separamos las oraciones y las labels

In [96]:
sentences = datos['sentence'].tolist()
labels = datos['label'].tolist()

## Creamos un Training y Test Set

In [94]:
training = []
porcentaje_entrenamiento = 0.75
s = sentences.copy()
l = labels.copy()
for i in range(int(len(l)*porcentaje_entrenamiento)):
    n = randint(0, len(l) -1)
    training.append({'sentence': s.pop(n), 'label': l.pop(n)})
test = [{'sentence': s[i], 'labels': l[i]} for i in range(len(s))]

In [97]:
print(len(training))
print(len(test))
print(len(labels))

352
118
470


In [102]:
trainer = Trainer(
    model,
    args,
    train_dataset=training,
    eval_dataset=test,
    tokenizer=tokenizer
)

ModuleAttributeError: 'GPT2ForSequenceClassification' object has no attribute 'model_parallel'